<a href="https://colab.research.google.com/github/WesleyKhoiVo/ViViT-action/blob/main/src/data-preprocessing/UBI_splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! pip install npy_append_array

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from tqdm import tqdm
import numpy as np
from npy_append_array import NpyAppendArray as npaa
import gc

In [11]:
ROOT = '/content/drive/My Drive/Datasets/extracted'
shape_x, shape_y = (299354, 28, 28, 28), (299354, 1)

In [12]:
dtype = np.float32
mode = 'r+'
xmap = np.memmap(ROOT+'/x.npy',dtype=dtype, shape=shape_x, mode=mode)

In [13]:
def perm_to_file(arr, perm:np.ndarray, filename:str,batch_size=1e4):
    with npaa(filename) as x:
        start = 0
        loop_end = int(max(len(perm),batch_size))
        loop_start = int(min(len(perm),batch_size))
        loop = [i for i in range(loop_start, loop_end,int(batch_size))]
        loop.append(len(perm))
        print(loop)
        for i in tqdm(loop):
            end = min(i, perm.shape[0])
            x.append(arr[start:end])
            # print(start,end)
            start = end


## Unbalanced data

In [14]:
# splitting 70 10 20
np.random.seed(123)
y = np.load(ROOT+'/y.npy')

y_viol = np.argwhere(y == 1)
y_norm = np.argwhere(y == 0)

perm_viol  = np.random.permutation(y_viol.shape[0])
train_ratio_viol = len(perm_viol) // 10 * 7
val_ratio_viol   = train_ratio_viol + len(perm_viol) // 10 * 1

print(train_ratio_viol, val_ratio_viol)

perm_norm  = np.random.permutation(y_norm.shape[0])
train_ratio_norm = len(perm_norm) // 10 * 7
val_ratio_norm   = train_ratio_norm + len(perm_norm) // 10

print(train_ratio_norm, val_ratio_norm)



FileNotFoundError: ignored

In [ ]:
len(perm_norm)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xmap[8,5]*255.,cmap='gray')

In [ ]:
train_perm = np.concatenate((perm_viol[:train_ratio_viol], perm_norm[:train_ratio_norm]))
val_perm   = np.concatenate((perm_viol[train_ratio_viol:val_ratio_viol], perm_norm[train_ratio_norm:val_ratio_norm]))
test_perm  = np.concatenate((perm_viol[val_ratio_viol:], perm_norm[val_ratio_norm:]))

np.random.shuffle(train_perm)
np.random.shuffle(val_perm)
np.random.shuffle(test_perm)

In [ ]:
# perm_to_file(xmap, val_perm,ROOT+'/x_val.npy',batch_size=5e4)
# perm_to_file(xmap, test_perm,ROOT+'/x_test.npy',batch_size=5e4)
perm_to_file(xmap, train_perm,ROOT+'/x_train.npy',batch_size=5e4)

In [ ]:
np.save(ROOT+'/y_val'   ,y[val_perm])
np.save(ROOT+'/y_test'  ,y[test_perm])
np.save(ROOT+'/y_train' ,y[train_perm])

In [ ]:
!ls {ROOT} -l

## Balanced data


In [ ]:
# ! rm -rf {ROOT}/balanced
# ! mkdir {ROOT}/balanced

In [ ]:
OUT = '/content/drive/My Drive/Datasets/extracted/balanced'
shape_x, shape_y = (299354, 28, 28, 28), (299354, 1)

# splitting 70 10 20
np.random.seed(123)
y = np.load(ROOT+'/y.npy')

y_viol = np.argwhere(y == 1)
y_norm = np.argwhere(y == 0)

size = min(y_viol.shape[0], y_norm.shape[0])

perm_viol  = np.random.permutation(y_viol.shape[0])
perm_viol  = perm_viol[:size]
train_ratio_viol = len(perm_viol) // 10 * 7
val_ratio_viol   = train_ratio_viol + len(perm_viol) // 10 * 1

print(train_ratio_viol, val_ratio_viol)

perm_norm  = np.random.permutation(y_norm.shape[0])
perm_norm  = perm_norm[:size]
train_ratio_norm = len(perm_norm) // 10 * 7
val_ratio_norm   = train_ratio_norm + len(perm_norm) // 10

print(train_ratio_norm, val_ratio_norm)

train_perm = np.concatenate((perm_viol[:train_ratio_viol], perm_norm[:train_ratio_norm]))
val_perm   = np.concatenate((perm_viol[train_ratio_viol:val_ratio_viol], perm_norm[train_ratio_norm:val_ratio_norm]))
test_perm  = np.concatenate((perm_viol[val_ratio_viol:], perm_norm[val_ratio_norm:]))

np.random.shuffle(train_perm)
np.random.shuffle(val_perm)
np.random.shuffle(test_perm)

In [ ]:
# perm_to_file(xmap, val_perm     ,OUT+'/x_val.npy'  ,batch_size=5e3)
# perm_to_file(xmap, test_perm    ,OUT+'/x_test.npy' ,batch_size=5e3)
perm_to_file(xmap, train_perm   ,OUT+'/x_train.npy',batch_size=5e3)

In [ ]:
np.save(OUT+'/y_val'   ,y[val_perm])
np.save(OUT+'/y_test'  ,y[test_perm])
np.save(OUT+'/y_train' ,y[train_perm])

In [ ]:
print(train_perm.shape,val_perm.shape,test_perm.shape)